# Quiz : 호랑이, 사자 이미지 분류

In [4]:
import os
import numpy as np
import pandas as pd
import math

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input

## sklearn 의 분류기를 케라스 분류기로 인식해줌
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# model 불러오기
from tensorflow.keras.models import load_model
# 이미지 어쩌고 모델
from tensorflow.keras.applications import vgg16
# from keras.applications import VGG16

# 시각화
import matplotlib.pyplot as plt
import seaborn as sb
import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False
%matplotlib inline


In [5]:
dataGen = image.ImageDataGenerator(rescale=1./255,)
bsize = 400
imgGen = dataGen.flow_from_directory('image/', 
                                     target_size=(98,98), batch_size=bsize)

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'image/'

In [ ]:
# 이미지 개수
n_img = imgGen.n
steps = n_img // bsize
steps

In [ ]:
# 
while True:
    try:
        x_data, y_data = imgGen.next()
        if x_data.shape[0] >= bsize:
            break
        else:
            continue
    except:
        print('.',end=' ')

x_data.shape, y_data.shape

In [ ]:
imgGen.class_indices

In [ ]:
plt.imshow(x_data[0])
plt.show()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data,
                                                    test_size=0.2,
                                                    stratify=y_data)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

## 전이학습

In [ ]:
# ResNet50 model 의 특징 추출기까지만 불러온다
model_conv = vgg16(weights='imagenet',
                      include_top=False,
                      input_shape=(98,98,3))
# model_conv.summary()

In [ ]:
# 특징 추출기를 학습 제외시키기
for layer in model_conv.layers:
    layer.trainable = False

In [ ]:
# model 학습
model = Sequential()
model.add(model_conv)
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['acc'])
h = model.fit(x_train, y_train, epochs=30)

In [ ]:
plt.plot(h.history['loss'], label='loss')
plt.plot(h.history['acc'], label='acc')
plt.legend()
plt.show()

In [ ]:
# 정확도 확인

labels = imgGen.class_indices.keys()
n_train = len(y_train)
n_test = len(y_test)

# train set
print('<<Train Set>> n:', n_train )
train_pred = model.predict(x_train).argmax(axis=1)
y_train_mx = y_train.argmax(axis=1)
train_conf = confusion_matrix(y_train_mx, train_pred)
print('Accuracy \t', accuracy_score(y_train_mx, train_pred))
print('F1 \t\t', f1_score(y_train_mx, train_pred, average='macro'))
print('Confusion matrix')
plt.figure()
sb.heatmap(train_conf, annot=True,cmap='Reds', fmt='d',
           xticklabels=labels, yticklabels=labels)
plt.show()

print('='*50)
# test set
print('<<Test Set>> n:', n_test)
test_pred = model.predict(x_test).argmax(axis=1)
y_train_mx = y_test.argmax(axis=1)
test_conf = confusion_matrix(y_train_mx, test_pred)
print('Accuracy \t', accuracy_score(y_train_mx, test_pred))
print('F1 \t\t', f1_score(y_train_mx, test_pred, average='macro'))
print('Confusion matrix')
plt.figure()
sb.heatmap(test_conf, annot=True,cmap='Reds', fmt='d',
           xticklabels=labels, yticklabels=labels)
plt.show()